In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from sklearn.ensemble import IsolationForest

import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

In [2]:
df = pd.read_csv('../input/dataset.csv')
sub = pd.read_csv('../input/submission.csv')
df.head()

,WindNumber,Time,WindSpeed,Power,RotorSpeed
0,1,2017/11/1 0:20,4.188333,65.750000,5.779167
1,1,2017/11/1 0:30,4.042500,120.700000,8.636500
2,1,2017/11/1 0:40,4.137368,135.157895,8.668421
3,1,2017/11/1 0:50,4.274737,157.315789,8.727895
4,1,2017/11/1 1:00,4.108947,129.894737,8.656842


In [3]:
df['Power_overload'] = np.where(df['Power'] > 2000, 1, 0)

In [4]:
def RotorSpeed_process(x):
    if x['WindNumber'] in [1, 2, 3, 4, 6, 7, 8, 9, 10] and x['RotorSpeed'] > 16.8:
        return 1
    elif x['WindNumber'] == 5 and x['RotorSpeed'] > 19:
        return 1
    elif x['WindNumber'] == 11 and x['RotorSpeed'] > 14:
        return 1
    elif x['WindNumber'] == 12 and x['RotorSpeed'] > 17:
        return 1
    else:
        return 0

df['RotorSpeed_overload'] = df.apply(RotorSpeed_process, axis=1)

In [5]:
df.drop(['Time', 'WindNumber'], axis=1, inplace=True)

In [6]:
clf = IsolationForest(n_estimators=100,
                      random_state=2020,
                      contamination='auto',
                      n_jobs=-1)
clf.fit(df)
y_pred = clf.predict(df)
y_pred = np.where(y_pred == -1, 1, 0)
df['label1'] = y_pred

In [7]:
df['label2'] = df['RotorSpeed'].apply(lambda x: 1 if x < 1 or x > 20 else 0)
df['label'] = df.apply(lambda x: 1 if x['label1'] == 1 or x['label2'] == 1 else 0, axis=1)
sub['label'] = df['label'].values
sub.to_csv('../sub/sub_rule.csv', index=False)